# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [1]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [2]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [3]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
slim_raw_df.head(3)

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours",{3: 'G G/B ...,"['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),{1: 'Em D C C...,"['canadian pop', 'pop', 'post-teen pop']",100


In [23]:
split_raw_df = slim_raw_df.copy()
splits = []

for row in split_raw_df.chords:
    row_dict = ast.literal_eval(row)
    for key, value in row_dict.items():
        r_or = value.split('|')[0]
        etc1 = r_or.replace('.', ' ')
        ect2 = etc1.replace('*', '')
        p = re.sub(r'[()]', '', ect2)
        s = re.sub(r'^\s+|\s+$|/', '', p)
        row_dict[key] = re.sub(r'\s+', ' ', s)
    splits.append(row_dict)

split_raw_df.chords = splits
len(splits)

135783

In [24]:
r = random.randint(0, 135782)
print(slim_raw_df.chords[r])
print(split_raw_df.chords[r])

{0: '\n\t  \t\tIntro: G# G#maj7 Bbm7 Eb7  ', 2: '        G#    G#maj7 G#  ', 4: '        G#maj7      Bbm7          Eb7    Bbm7 Eb7   ', 6: 'Bbm7                 Eb7           ', 8: 'G#       Aº    Bbm7           Eb7   ', 10: 'G#       G#maj7 G#   ', 12: '          G#maj7    Bbm7   ', 14: '      Eb7      Bbm7 Eb7   ', 16: 'Bbm7    Eb7  Bbm7          Eb7  ', 18: 'G#             G#7   ', 20: '  C#maj7      C#m7  ', 22: '          F#7              Cm7   ', 24: 'F7                      Bbm7   ', 26: 'Eb7                  G#        G#7  ', 28: '        C#maj7 C#m7   ', 30: '          F#7                 Cm7   ', 32: 'F7                 Bbm7   ', 34: 'Eb7                  G#                G#7  ', 39: 'Bbm7 Eb7 Bbm7 Eb7 Bbm7 Eb7 Bbm7 Eb7 G# G#7  ', 41: '  C#maj7      C#m7  ', 43: '          F#7              Cm7   ', 45: 'F7                      Bbm7   ', 47: 'Eb7                  G#        G#7  ', 49: '        C#maj7 C#m7   ', 51: '          F#7                 Cm7   ', 53: 'F7               

#### See Chord Frequency

In [43]:
chords_counter = Counter()
for song in splits:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

28231

In [54]:
letters = list(string.ascii_uppercase)[:7]
non_chord_filter = [chord for chord in sorted_cc_df.index if chord[0] in letters]
cc_df = sorted_cc_df[sorted_cc_df.index.isin(non_chord_filter)]
cc_df.drop(index=['Bridge', 'Bridge:'], inplace=True)

pd.set_option('display.max_rows', 1190)
# sorted_cc_df.head(70*17)
cc_df.iloc[:500]

/var/folders/27/7y2sywgx471112_cg02mrd300000gn/T/ipykernel_629/3919734243.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_df.drop(index=['Bridge', 'Bridge:'], inplace=True)


,chord_count
G,1064777
C,862667
D,826922
A,661687
F,519077
E,479468
Am,443690
Em,398376
Bm,247095
B,239157


In [ ]:
# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

## Creating List of Correctly Formatted Chords

In [ ]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [ ]:
chords_df = pd.DataFrame(index=['m', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', '7', 'M7',  
                                'm9', '9', '7b9',  'M9', '11',
                                'add2', 'add4', 'add6', 'add9', 'madd2', 'madd4', 'madd6', 'madd9', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4'
                                ],
                         columns=major_chords)


In [ ]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df.head(3)

In [ ]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 
                'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 
                'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']
slash_indexes = ['m/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/7', 'm/M7',
                'm7/b3', 'm7/4', 'm7/5', 'm7/M7',
                '7/2', '7/b3', '7/3', '7/4', '7/5', '7/6','7/M7', 
                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/M7',
                '/2', '/b3', '/3', '/4', '/5', '/6','/7', '/M7', 
                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/7']
whole_to_half = {'2': 2, 'b3': 3, '3': 4, '4': 5, '5': 7, '6': 9, '7': 10, 'M7': 11}

In [ ]:
# big function
def get_slash_notes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')[1]
            h = whole_to_half[s]
            sc = f'{chord}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if n in chord :
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
            elif len(chord) > 1:
                if '#' == chord[1]:
                    i = sharp_notes.index(chord[:2])
                    find_slash(chord, sharp_notes, i)
                elif 'b' == chord[1]:
                    i = flats_notes.index(chord[:2])
                    find_slash(chord, flats_notes, i)
    
    for chord in columns:
        sort_f_s(chord, sharp_notes)
        sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [ ]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
slash_chords_df

## Shove chords into correct format

#### For next time:
- merge chords
- delete everything other than those